# CSML1020 Course Project - New Plant Diseases Dectection
## Authors (Group 3): Paul Doucet, Jerry Khidaroo

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Initialization

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os

In [ ]:
# adjust pandas display
pd.options.display.max_columns = 30
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.precision = 2
pd.options.display.max_colwidth = -1

In [ ]:
# Import matplotlib and seaborn and adjust some defaults
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 100

import seaborn as sns
sns.set_style("whitegrid")

## Data Preparation

In [ ]:
# List of all directories and file counts in given path
def getDirCounts(dirName):
    listOfFile = os.listdir(dirName)
    rows = []
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            fields = entry.split("___")
            if fields[1] == 'healthy':
                status = 'healthy'
            else:
                status = "unhealthy"
            
            disease = fields[1].split("_")[-1].replace(")","").lower()
            
            noOfFiles = sum([len(files) for r, d, files in os.walk(fullPath)])
            rows.append([fields[0], fields[1], noOfFiles, status, disease])

    df = pd.DataFrame(rows, columns=["plant", "condition", "count", "status", "disease"])
    #df['plant_status'] = df['plant'] + ' - ' + df['status']
    return df

In [ ]:
dataDirTrain = '../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
dataDirValidate = '../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'
dataDirTest = '../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test'

In [ ]:
!ls

In [ ]:
# Get the list of all files in directory tree at given path
plants_df = getDirCounts(dataDirTrain)
plants_df

## Data Importing

In [ ]:
import tensorflow as tf


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

In [ ]:
from keras.applications.vgg16 import VGG16
import keras
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

## Data Pre-Processing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

# batch_size = 128
batch_size = 48
training_set = train_datagen.flow_from_directory(dataDirTrain,
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

valid_set = valid_datagen.flow_from_directory(dataDirValidate,
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')

In [ ]:
class_dict = training_set.class_indices
print(class_dict)

In [ ]:
li = list(class_dict.keys())
print(li)

In [ ]:
train_num = training_set.samples
valid_num = valid_set.samples

---

In [ ]:
def get_model():
    base_model=ResNet50(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    classifier.add(Dense(38,activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [ ]:
def Save_Benchmark(descr, metric, reset_rb, init_rb):
    global rows_benchmarks
    #global rows_benchmarks_all
    global df_benchmarks
    #global df_benchmarks_all
    # if (rows_benchmarks == None):
    #     rows_benchmarks = []
    if (init_rb):
        rows_benchmarks = []
    else:
        if (reset_rb):
            rows_benchmarks = []

        # if (reset_rb_all):
        #     rows_benchmarks_all = []
        rows_benchmarks.append([descr, metric])
        #rows_benchmarks_all.append([descr, metric])
        df_benchmarks = pd.DataFrame(rows_benchmarks, columns=["Preprosessing Steps", "accuracy"])
        #df_benchmarks_all = pd.DataFrame(rows_benchmarks_all, columns=["Preprosessing Steps", "accuracy"])


In [ ]:
def run_model(train_datagen, valid_set, batch_size, epochs):    
    training_set = train_datagen.flow_from_directory(dataDirTrain, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

    train_num = training_set.samples
    valid_num = valid_set.samples

    classifier = get_model()
    
    #fitting images to CNN
    history = classifier.fit(training_set, steps_per_epoch=train_num//batch_size, validation_data=valid_set, epochs=epochs, validation_steps=valid_num//batch_size)

    return history

---

# Model Preparation VGG16

In [ ]:
base_model=VGG16(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

In [ ]:
classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
classifier.add(Dense(38,activation='softmax'))
classifier.summary()

In [ ]:
dot_img_file = '/tmp/model_VGG16.png'
tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)

In [ ]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [ ]:
#classifier.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

In [ ]:
#fitting images to CNN
#history = classifier.fit(training_set,
#                         steps_per_epoch=train_num//batch_size,
#                         validation_data=valid_set,
#                         epochs=3,
#                         validation_steps=valid_num//batch_size,
#                         )

In [ ]:
#Saving our model
#filepath="./Mymodel_VGG16.h5"
#classifier.save(filepath)

In [ ]:
def get_model():
    base_model=VGG16(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    classifier.add(Dense(38,activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [ ]:
def Save_Benchmark(descr, metric, reset_rb, init_rb):
    global rows_benchmarks
    #global rows_benchmarks_all
    global df_benchmarks
    #global df_benchmarks_all
    # if (rows_benchmarks == None):
    #     rows_benchmarks = []
    if (init_rb):
        rows_benchmarks = []
    else:
        if (reset_rb):
            rows_benchmarks = []

        # if (reset_rb_all):
        #     rows_benchmarks_all = []
        rows_benchmarks.append([descr, metric])
        #rows_benchmarks_all.append([descr, metric])
        df_benchmarks = pd.DataFrame(rows_benchmarks, columns=["Preprosessing Steps", "accuracy"])
        #df_benchmarks_all = pd.DataFrame(rows_benchmarks_all, columns=["Preprosessing Steps", "accuracy"])


In [ ]:
def run_model(train_datagen, valid_set, batch_size, epochs):    
    training_set = train_datagen.flow_from_directory(dataDirTrain, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

    train_num = training_set.samples
    valid_num = valid_set.samples

    classifier = get_model()
    
    #fitting images to CNN
    history = classifier.fit(training_set, steps_per_epoch=train_num//batch_size, validation_data=valid_set, epochs=epochs, validation_steps=valid_num//batch_size)

    return history

In [ ]:
dg_descrs = ["rescale=1./255", 
    #"rescale=1./255, shear_range=0.2", 
    #"rescale=1./255, zoom_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2",
    #"rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2",
    #"rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0]"
]

datagens = [
    ImageDataGenerator(rescale=1./255, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, width_shift_range=0.2, fill_mode='nearest'),
    #mageDataGenerator(rescale=1./255, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0], fill_mode='nearest'),
]


batch_size = 48
epochs = 10
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
Save_Benchmark("", "", False, True)

for train_datagen, dg_descr in zip(datagens, dg_descrs):
    history = run_model(train_datagen, valid_set, batch_size, epochs)
    #history.history
    Save_Benchmark(dg_descr, history.history['val_accuracy'][2], False, False)

df_benchmarks

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Inference - VGG16

In [ ]:
# predicting an image
from keras.preprocessing import image
import numpy as np
image_path = "../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test/CornCommonRust1.JPG"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = classifier.predict(img)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = li[index]

#ploting image with predicted class name        
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(class_name)
plt.show()

In [ ]:
pred = classifier.predict(valid_set)

---

# Model Preparation ResNet50

In [ ]:
# example of loading the resnet50 model
from keras.applications.resnet50 import ResNet50

In [ ]:
base_model=ResNet50(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

In [ ]:
classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
classifier.add(Dense(38,activation='softmax'))
classifier.summary()

In [ ]:
dot_img_file = '/tmp/model_resnet50.png'
tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)

In [ ]:
#classifier.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

In [ ]:
#fitting images to CNN
#history = classifier.fit(training_set,
#                         steps_per_epoch=train_num//batch_size,
#                         validation_data=valid_set,
#                         epochs=3,
#                         validation_steps=valid_num//batch_size,
#                         )

In [ ]:
#Saving our model
#filepath="./Mymodel_sample_resnet50.h5"
#classifier.save(filepath)

In [ ]:
def get_model():
    base_model=ResNet50(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    classifier.add(Dense(38,activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [ ]:
def Save_Benchmark(descr, metric, reset_rb, init_rb):
    global rows_benchmarks
    #global rows_benchmarks_all
    global df_benchmarks
    #global df_benchmarks_all
    # if (rows_benchmarks == None):
    #     rows_benchmarks = []
    if (init_rb):
        rows_benchmarks = []
    else:
        if (reset_rb):
            rows_benchmarks = []

        # if (reset_rb_all):
        #     rows_benchmarks_all = []
        rows_benchmarks.append([descr, metric])
        #rows_benchmarks_all.append([descr, metric])
        df_benchmarks = pd.DataFrame(rows_benchmarks, columns=["Preprosessing Steps", "accuracy"])
        #df_benchmarks_all = pd.DataFrame(rows_benchmarks_all, columns=["Preprosessing Steps", "accuracy"])


In [ ]:
def run_model(train_datagen, valid_set, batch_size, epochs):    
    training_set = train_datagen.flow_from_directory(dataDirTrain, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

    train_num = training_set.samples
    valid_num = valid_set.samples

    classifier = get_model()
    
    #fitting images to CNN
    history = classifier.fit(training_set, steps_per_epoch=train_num//batch_size, validation_data=valid_set, epochs=epochs, validation_steps=valid_num//batch_size)

    return history

In [ ]:
dg_descrs = ["rescale=1./255", 
    "rescale=1./255, shear_range=0.2", 
    #"rescale=1./255, zoom_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2",
    #"rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2",
    #"rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0]"
]

datagens = [
    ImageDataGenerator(rescale=1./255, fill_mode='nearest'),
    ImageDataGenerator(rescale=1./255, shear_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, width_shift_range=0.2, fill_mode='nearest'),
    #mageDataGenerator(rescale=1./255, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0], fill_mode='nearest'),
]


batch_size = 48
epochs = 5
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
Save_Benchmark("", "", False, True)

for train_datagen, dg_descr in zip(datagens, dg_descrs):
    history = run_model(train_datagen, valid_set, batch_size, epochs)
    #history.history
    Save_Benchmark(dg_descr, history.history['val_accuracy'][2], False, False)

df_benchmarks

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Inference - ResNET50

In [ ]:
# predicting an image
from keras.preprocessing import image
import numpy as np
image_path = "../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test/CornCommonRust1.JPG"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = classifier.predict(img)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = li[index]

#ploting image with predicted class name        
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(class_name)
plt.show()

In [ ]:
pred_resnet_50 = classifier.predict(valid_set)

---

# Model Preparation InceptionV3

In [ ]:
# example of loading the InceptionV3 model
from keras.applications import InceptionV3

In [ ]:
base_model=InceptionV3(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

In [ ]:
base_model=InceptionV3(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
classifier.add(Dense(38,activation='softmax'))
    
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
dot_img_file = '/tmp/model_InceptionV3.png'
tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)
    


In [ ]:
def get_model():
    base_model=InceptionV3(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    classifier.add(Dense(38,activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    dot_img_file = '/tmp/model_InceptionV3.png'
    tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)
    
    return classifier

In [ ]:
dg_descrs = ["rescale=1./255", 
    #"rescale=1./255, shear_range=0.2", 
    #"rescale=1./255, zoom_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2",
    #"rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2",
    #"rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0]"
]

datagens = [
    ImageDataGenerator(rescale=1./255, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, width_shift_range=0.2, fill_mode='nearest'),
    #mageDataGenerator(rescale=1./255, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0], fill_mode='nearest'),
]


batch_size = 48
epochs = 10
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
Save_Benchmark("", "", False, True)

for train_datagen, dg_descr in zip(datagens, dg_descrs):
    history = run_model(train_datagen, valid_set, batch_size, epochs)
    #history.history
    Save_Benchmark(dg_descr, history.history['val_accuracy'][2], False, False)

df_benchmarks

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Inference - InceptionV3

In [ ]:
# predicting an image
from keras.preprocessing import image
import numpy as np
image_path = "../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test/CornCommonRust1.JPG"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = classifier.predict(img)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = li[index]

#ploting image with predicted class name        
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(class_name)
plt.show()

In [ ]:
pred_inceptV3 = classifier.predict(valid_set)

# Review of a Model File

In [ ]:
# Python program to demonstrate 
# HDF5 file 
  
import numpy as np 
import h5py 


In [ ]:
# initializing a random numpy array 
arr = np.random.randn(1000) 


In [ ]:
# creating a file 
with h5py.File('test.hdf5', 'w') as f:  
    dset = f.create_dataset("default", data = arr) 
    

In [ ]:
# open the file as 'f' 
with h5py.File('../input/trialh5view/Mymodel_VGG16.h5', 'r') as f:  
    data = f['default'] 
      
    # get the minimum value 
    print(min(data))  
      
    # get the maximum value 
    print(max(data)) 
      
    # get the values ranging from index 0 to 15 
    print(data[:15]) 

In [ ]:
! ls ../output

---

## View Weights

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# example of loading the InceptionV3 model
from keras.applications import InceptionV3

In [ ]:
base_model=InceptionV3(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

---

## Xception

In [ ]:
# example of loading the Xception model
from keras.applications import Xception

In [ ]:
base_model=Xception(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

In [ ]:
base_model=Xception(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
classifier.add(Dense(38,activation='softmax'))
    
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
dot_img_file = '/tmp/model_InceptionV3.png'
tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)
    

In [ ]:
def get_model():
    base_model=Xception(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    classifier.add(Dense(38,activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    dot_img_file = '/tmp/model_InceptionV3.png'
    tf.keras.utils.plot_model(classifier, to_file=dot_img_file, show_shapes=True)
    
    return classifier

In [ ]:
dg_descrs = ["rescale=1./255", 
    #"rescale=1./255, shear_range=0.2", 
    #"rescale=1./255, zoom_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2", 
    #"rescale=1./255, width_shift_range=0.2",
    #"rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2",
    #"rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0]"
]

datagens = [
    ImageDataGenerator(rescale=1./255, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, width_shift_range=0.2, fill_mode='nearest'),
    #mageDataGenerator(rescale=1./255, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest'),
    #ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0], zoom_range=[0.5,1.0], fill_mode='nearest'),
]


batch_size = 48
epochs = 10
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
Save_Benchmark("", "", False, True)

for train_datagen, dg_descr in zip(datagens, dg_descrs):
    history = run_model(train_datagen, valid_set, batch_size, epochs)
    #history.history
    Save_Benchmark(dg_descr, history.history['val_accuracy'][2], False, False)

df_benchmarks

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

![](http://)## Inference - InceptionV3

In [ ]:
# predicting an image
from keras.preprocessing import image
import numpy as np
image_path = "../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test/CornCommonRust1.JPG"
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255

print("Following is our prediction:")
prediction = classifier.predict(img)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = li[index]

#ploting image with predicted class name        
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(class_name)
plt.show()